За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за посиланням (буде завантажено наступним кроком). Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою. Використайте метод classification report для порівняння. Порівняння моделей на основі однієї метрики(такої як Accuracy)- не приймається.

## Крок 1: Завантаження та огляд датасету

In [15]:
import pandas as pd
import zipfile
import os
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Path to the zip file on Google Drive
zip_file_path = '/content/drive/MyDrive/Files/homework.zip'
extracted_dir_path = '/content/homework_extracted/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

In [4]:
import pandas as pd

# Function to read CSV files and add a label
def read_and_label_csv(file_path, label):
    df = pd.read_csv(file_path)
    df['label'] = label
    return df

In [5]:
# Corrected mapping of directories to labels
label_mapping = {
    'idle': 'stand',
    'walking': 'walk',
    'running': 'run',
    'stairs': 'stairs'
}

# Read all files and combine them into one dataframe
dataframes = []
for label_dir, label in label_mapping.items():
    dir_path = os.path.join(extracted_dir_path, 'data', label_dir)
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        dataframes.append(read_and_label_csv(file_path, label))

# Combine all dataframes into one
combined_df = pd.concat(dataframes, ignore_index=True)

In [6]:
combined_df.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,label
0,0.421380,-0.090980,9.730036,stand
1,0.392649,-0.019154,9.753978,stand
2,0.392649,-0.033519,9.763555,stand
3,0.387861,-0.043096,9.734824,stand
4,0.383072,-0.019154,9.725247,stand


## Крок 2: Підготовка даних та розрахунок часових ознак
Часові ознаки: це можуть бути такі показники, як середнє значення, стандартне відхилення, максимальне та мінімальне значення акселерометра за певний період часу.

In [7]:
# Function to calculate time domain features
def calculate_time_domain_features(df, window_size=100):
    features = []
    labels = []

    for i in range(0, len(df) - window_size + 1, window_size):
        window = df.iloc[i:i + window_size]
        feature = {
            'mean_x': window['accelerometer_X'].mean(),
            'mean_y': window['accelerometer_Y'].mean(),
            'mean_z': window['accelerometer_Z'].mean(),
            'std_x': window['accelerometer_X'].std(),
            'std_y': window['accelerometer_Y'].std(),
            'std_z': window['accelerometer_Z'].std(),
            'max_x': window['accelerometer_X'].max(),
            'max_y': window['accelerometer_Y'].max(),
            'max_z': window['accelerometer_Z'].max(),
            'min_x': window['accelerometer_X'].min(),
            'min_y': window['accelerometer_Y'].min(),
            'min_z': window['accelerometer_Z'].min()
        }
        features.append(feature)
        labels.append(window['label'].mode()[0])

    features_df = pd.DataFrame(features)
    features_df['label'] = labels

    return features_df

In [8]:
# Calculate time domain features for the combined dataframe
features_df = calculate_time_domain_features(combined_df)

# Display the first few rows of the features dataframe
features_df.head()


,mean_x,mean_y,mean_z,std_x,std_y,std_z,max_x,max_y,max_z,min_x,min_y,min_z,label
0,0.237170,0.042569,9.762405,0.229005,0.126020,0.031048,0.560243,0.354342,9.945514,-0.234632,-0.205901,9.720459,stand
1,0.273466,0.120045,9.789843,0.334984,1.063113,0.437549,1.292869,6.641516,12.248735,-2.504335,-2.308010,7.958327,stand
2,0.136422,2.166465,9.130719,0.573474,2.779970,1.060216,4.118027,7.201759,11.530476,-1.570596,-0.186748,6.359000,stand
3,-0.118034,2.227230,9.002869,0.518866,2.737829,1.046330,2.341529,7.015011,10.515334,-1.561019,-0.995988,6.129156,stand
4,0.137954,6.157312,7.462248,0.548519,0.956433,0.819090,2.020706,7.699753,9.260772,-0.823605,4.007894,5.908890,stand


## Крок 3: Розділення даних на тренувальний та тестовий набори.

In [11]:
# Split the data into training and testing sets
X = features_df.drop(columns=['label'])
y = features_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f'Training set size: {X_train.shape[0]}')
print(f'Test set size: {X_test.shape[0]}')

Training set size: 1356
Test set size: 582


## Крок 4: Навчання моделей SVM (Support Vector Machine): Використаємо SVC з бібліотеки scikit-learn.

In [18]:
# Train SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Make predictions
y_pred_svm = svm_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

         run       1.00      1.00      1.00       309
      stairs       0.94      1.00      0.97        15
       stand       1.00      0.99      0.99        91
        walk       0.99      0.99      0.99       167

    accuracy                           1.00       582
   macro avg       0.98      1.00      0.99       582
weighted avg       1.00      1.00      1.00       582



## Крок 4: Навчання моделей Випадковий ліс (Random Forest): Використаємо RandomForestClassifier з бібліотеки scikit-learn.

In [19]:
# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

         run       1.00      1.00      1.00       309
      stairs       1.00      1.00      1.00        15
       stand       1.00      0.99      0.99        91
        walk       0.99      1.00      1.00       167

    accuracy                           1.00       582
   macro avg       1.00      1.00      1.00       582
weighted avg       1.00      1.00      1.00       582



## Крок 5: Порівняння результатів
Використаємо метод classification_report для отримання метрик, таких як precision, recall, f1-score для кожної моделі.

In [20]:
# Evaluate the SVM model
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))

# Evaluate the Random Forest model
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

SVM Classification Report:
              precision    recall  f1-score   support

         run       1.00      1.00      1.00       309
      stairs       0.94      1.00      0.97        15
       stand       1.00      0.99      0.99        91
        walk       0.99      0.99      0.99       167

    accuracy                           1.00       582
   macro avg       0.98      1.00      0.99       582
weighted avg       1.00      1.00      1.00       582

Random Forest Classification Report:
              precision    recall  f1-score   support

         run       1.00      1.00      1.00       309
      stairs       1.00      1.00      1.00        15
       stand       1.00      0.99      0.99        91
        walk       0.99      1.00      1.00       167

    accuracy                           1.00       582
   macro avg       1.00      1.00      1.00       582
weighted avg       1.00      1.00      1.00       582



## Висновки

1. **Точність (Accuracy)**: Обидві моделі мають дуже високу точність (1.00), що означає, що обидві моделі правильно класифікували всі приклади у тестовому наборі.
   
2. **Precision та Recall**:
   - Модель SVM має трохи нижчу precision для класу "stairs" (0.94) у порівнянні з Random Forest (1.00).
   - Всі інші класи мають однакові показники precision та recall для обох моделей.

3. **F1-score**:
   - Для класу "stairs" F1-score у SVM моделі (0.97) трохи нижчий, ніж у Random Forest (1.00). Це відображає різницю у precision для цього класу.

4. **Macro avg та Weighted avg**:
   - Обидві моделі мають дуже високі середні значення macro avg та weighted avg для всіх трьох метрик (precision, recall, f1-score).

## Рекомендації

Обидві моделі показують відмінні результати. Random Forest демонструє незначно кращі показники для класу "stairs". В іншому випадку, обидві моделі можуть використовуватися залежно від інших вимог (наприклад, обчислювальна ефективність).